In [1]:
import copy
import pygame
import numpy as np
import matplotlib.pyplot as plt

pygame 2.5.2 (SDL 2.28.3, Python 3.12.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Forest Fires

In [4]:
pygame.init()

width = 400
height = 400
ticks = 1000
screen = pygame.display.set_mode((width, height))
clock  = pygame.time.Clock()
font = pygame.font.Font(None, 24)
data = []

# initialize game
regrowth_chance = 0.01
lightning_chance = 0.0001
spreading_chance = 0.5

length = min(width, height) // 100
forest = [['empty' for col in range(100)] for row in range(100)]

for row in range(100):
    for col in range(100):
        if np.random.random() < 0.5:
            forest[row][col] = 'tree'
   

running = True
for tick in range(ticks):

    # set the frames per second FPS
    clock.tick(20)

    # set the initial screen color in RGB
    screen.fill((255, 255, 255))

    # copy for simultanious update
    new_forest = copy.deepcopy(forest)

    # update 
    for row in range(100):
        for col in range(100):
            state = forest[row][col]

            # regrowth
            if state == 'empty':
                if np.random.random() < regrowth_chance:
                    new_forest[row][col] = 'tree'

            # lightning
            if state == 'tree':
                if np.random.random() < lightning_chance:            
                    new_forest[row][col] = 'fire'

            # spread
            if state == 'fire':
                for i in range(-1, 2):
                    for j in range(-1, 2):
                        nrow = row + i
                        ncol = col + j
                        if nrow >= 0 and ncol >= 0:
                            if nrow <= 99 and ncol <= 99:
                                neighbor = forest[nrow][ncol]
                                if neighbor == 'tree':
                                    if np.random.random() < spreading_chance:
                                        new_forest[nrow][ncol] = 'fire'
                new_forest[row][col] = 'empty'
                

    # copy for simultanious update
    forest = copy.deepcopy(new_forest)
    
    # draw update
    for row in range(100):
        for col in range(100):
            state = forest[row][col]
            if state == 'tree':
                color = (0, 255, 0)
            if state == 'fire':
                color = (255, 0, 0)
            if state == 'empty':
                color = (75, 75, 0)
            pygame.draw.rect(screen, color, (col*length, row*length, length, length))
            

    # display and update the current tick
    text = font.render(str(tick), True, (0, 0, 0))
    screen.blit(text, (10, 10))
    
    # update the whole screen
    pygame.display.flip()

    # store data for each tick
    num_of_trees = 0
    num_of_fires = 0
    num_of_empty = 0
    for row in range(100):
        for col in range(100):
            state = forest[row][col]
            if state == 'tree':
                num_of_trees += 1
            if state == 'fire':
                num_of_fires += 1
            if state == 'empty':
                num_of_empty += 1
                
    data.append([num_of_trees, num_of_fires, num_of_empty])

    # handle the exit buttom
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            running = False
            
    # and break the loop
    if not running:
        break
    
pygame.quit()

In [12]:
trees, fires, empty = zip(*data)